# Scheduled Alerts - Testing Notebook

This notebook allows you to test the full scheduled alerts flow:
1. Subscribe to an alert
2. Generate real insights using Cortex Analyst or SQL
3. Send formatted emails

Works locally or in Snowflake Notebooks.


In [13]:
# Setup - Get Snowflake Session
import json
from datetime import datetime

try:
    # In Snowflake Notebook
    from snowflake.snowpark.context import get_active_session
    session = get_active_session()
    print("✅ Using Snowflake Notebook session")
except:
    # Local development
    import toml
    from pathlib import Path
    from snowflake.snowpark import Session
    
    config_path = Path.home() / '.snowflake' / 'config.toml'
    sf_config = toml.load(config_path)
    conn = sf_config.get('connections', {}).get('snowflake_agents', {})
    
    session = Session.builder.configs({
        'account': conn.get('account'),
        'user': conn.get('user'),
        'password': conn.get('password'),
        'database': 'SKI_RESORT_DB',
        'schema': 'AGENTS',
        'warehouse': conn.get('warehouse', 'COMPUTE_WH'),
    }).create()
    print("✅ Using local session")

print(f"Database: {session.get_current_database()}")


✅ Using Snowflake Notebook session
Database: "SKI_RESORT_DB"


In [14]:
# Configuration
TEST_EMAIL = "jeremy.demlow@snowflake.com"
EMAIL_INTEGRATION = "MY_EMAIL_INT"

print(f"Test Email: {TEST_EMAIL}")


Test Email: jeremy.demlow@snowflake.com


## Subscribe to an Alert


In [15]:
# Subscribe to an alert
QUESTION = "What were the average lift wait times yesterday?"
FREQUENCY = "Daily"

result = session.call('SKI_RESORT_DB.AGENTS.SCHEDULE_ALERT', TEST_EMAIL, QUESTION, FREQUENCY)
print(f"📬 {result}")


📬 INFO: You already have an active subscription for this question.


In [16]:
# View your alerts
alerts_json = session.call('SKI_RESORT_DB.AGENTS.VIEW_MY_ALERTS', TEST_EMAIL)
print("📋 Your Active Alerts:")
alerts_json


📋 Your Active Alerts:


'[{"alert_id":101,"created_at":"2025-12-03 16:50:19.105","frequency":"Daily","last_sent":"2025-12-03 16:53:04.407","question":"Daily lift wait time summary","times_sent":2},{"alert_id":201,"created_at":"2025-12-03 16:57:54.073","frequency":"Daily","last_sent":"Never","question":"What were the average lift wait times yesterday?","times_sent":0}]'

## Generate Real Insights from Data


In [17]:
# Query REAL lift wait time data
df = session.sql("""
SELECT 
    l.LIFT_NAME,
    ROUND(AVG(f.WAIT_TIME_MINUTES), 1) as AVG_WAIT,
    ROUND(MAX(f.WAIT_TIME_MINUTES), 1) as MAX_WAIT,
    COUNT(*) as TOTAL_SCANS
FROM SKI_RESORT_DB.MARTS.FACT_LIFT_SCANS f
JOIN SKI_RESORT_DB.MARTS.DIM_LIFT l ON f.LIFT_KEY = l.LIFT_KEY
JOIN SKI_RESORT_DB.MARTS.DIM_DATE d ON f.DATE_KEY = d.DATE_KEY
WHERE d.FULL_DATE >= DATEADD(day, -7, CURRENT_DATE())
GROUP BY l.LIFT_NAME
ORDER BY AVG_WAIT DESC
LIMIT 100
""").to_pandas()

print("📊 Lift Wait Times (Last 7 Days):")
print(df.to_string(index=False))

# Calculate summary stats
overall_avg = df['AVG_WAIT'].mean()
worst_lift = df.iloc[0]['LIFT_NAME']
worst_wait = df.iloc[0]['AVG_WAIT']

print(f"\n📈 Summary:")
print(f"   Overall average: {overall_avg:.1f} minutes")
print(f"   Longest waits: {worst_lift} ({worst_wait} min)")


📊 Lift Wait Times (Last 7 Days):
           LIFT_NAME  AVG_WAIT  MAX_WAIT  TOTAL_SCANS
        Mid Mountain       2.0       8.0         3466
          Family Fun       1.9       8.4         3329
            Sunshine       1.8       7.8         2474
  Backcountry Access       1.8       7.4         1848
            Blue Sky       1.8       7.9         2717
Terrain Park Express       1.8       7.7         2149
             Cruiser       1.8      10.2         3202
         South Ridge       1.8       8.1         2496
         Eagle Ridge       1.7       7.7         3078
           Kids Zone       1.7       7.8         1385
           West Bowl       1.7       7.6         1354
       Learners Lift       1.7       8.1         1171
         Powder Peak       1.7       7.4         1576
           East Side       1.7       7.6         2332
      Summit Express       1.6       9.3         4108
          North Face       1.6       5.9          723
       Beginner Bowl       1.6       6.8         

In [18]:
# Generate HTML email with REAL data
timestamp = datetime.now().strftime("%B %d, %Y at %I:%M %p")

# Build data table HTML
table_rows = ""
for _, row in df.head(5).iterrows():
    table_rows += f"<tr><td>{row['LIFT_NAME']}</td><td>{row['AVG_WAIT']} min</td><td>{row['MAX_WAIT']} min</td></tr>"

html_email = f"""
<!DOCTYPE html>
<html>
<head>
    <style>
        body {{ font-family: -apple-system, BlinkMacSystemFont, sans-serif; }}
        .container {{ max-width: 600px; margin: 20px auto; background: white; border-radius: 12px; box-shadow: 0 2px 8px rgba(0,0,0,0.1); }}
        .header {{ background: linear-gradient(135deg, #1e88e5, #1565c0); padding: 25px; color: white; border-radius: 12px 12px 0 0; }}
        .content {{ padding: 25px; }}
        .insights {{ background: #e3f2fd; padding: 20px; border-radius: 8px; border-left: 4px solid #1976d2; }}
        table {{ width: 100%; border-collapse: collapse; margin-top: 15px; }}
        th {{ background: #f5f5f5; padding: 10px; text-align: left; }}
        td {{ padding: 10px; border-bottom: 1px solid #eee; }}
        .footer {{ background: #fafafa; padding: 15px; font-size: 12px; color: #666; text-align: center; }}
    </style>
</head>
<body>
    <div class="container">
        <div class="header">
            <h1 style="margin:0; font-size:18px;">🎿 Daily Lift Wait Time Summary</h1>
            <div style="opacity:0.9; font-size:13px; margin-top:8px;">{timestamp}</div>
        </div>
        <div class="content">
            <div class="insights">
                <h3 style="margin-top:0; color:#1565c0;">📊 Key Insights</h3>
                <ul>
                    <li><strong>Overall average wait:</strong> {overall_avg:.1f} minutes</li>
                    <li><strong>Longest waits:</strong> {worst_lift} ({worst_wait} min avg)</li>
                    <li><strong>Best time to ski:</strong> Before 10am for shortest waits</li>
                </ul>
            </div>
            <h4>Top 5 Lifts by Wait Time</h4>
            <table>
                <tr><th>Lift</th><th>Avg Wait</th><th>Peak Wait</th></tr>
                {table_rows}
            </table>
        </div>
        <div class="footer">
            Ski Resort Analytics | Sent to: {TEST_EMAIL}
        </div>
    </div>
</body>
</html>
"""

print(f"✅ Generated email ({len(html_email)} chars)")


✅ Generated email (2252 chars)


In [12]:
# SEND the email with REAL data!
print(f"📤 Sending to {TEST_EMAIL}...")

try:
    escaped_html = html_email.replace("'", "''")
    session.sql(f"""
        CALL SYSTEM$SEND_EMAIL(
            '{EMAIL_INTEGRATION}',
            '{TEST_EMAIL}',
            '🎿 Daily Lift Wait Times - Real Data!',
            '{escaped_html}',
            'text/html'
        )
    """).collect()
    print(f"✅ Email sent to {TEST_EMAIL}!")
    print("   Check your inbox for the email with REAL lift wait time data.")
except Exception as e:
    print(f"❌ Error: {e}")


📤 Sending to jeremy.demlow@snowflake.com...
✅ Email sent to jeremy.demlow@snowflake.com!
   Check your inbox for the email with REAL lift wait time data.


---
## Alert Management

View, delete, and manage your alert subscriptions.


In [21]:
# View ALL alerts in the system (admin view)
all_alerts = session.sql("""
    SELECT 
        ALERT_ID,
        USER_EMAIL,
        OVERALL_QUESTION,
        ALERT_FREQUENCY,
        IS_ACTIVE,
        SEND_COUNT,
        LAST_SENT_AT,
        CREATED_AT
    FROM SKI_RESORT_DB.AGENTS.SCHEDULED_ALERTS
    ORDER BY CREATED_AT DESC
""").to_pandas()

print(f"📋 All Alerts in System ({len(all_alerts)} total):")
print(all_alerts.to_string(index=False))


📋 All Alerts in System (3 total):
 ALERT_ID                  USER_EMAIL                                 OVERALL_QUESTION ALERT_FREQUENCY  IS_ACTIVE  SEND_COUNT            LAST_SENT_AT              CREATED_AT
      201 jeremy.demlow@snowflake.com What were the average lift wait times yesterday?           Daily       True           0                     NaT 2025-12-03 16:57:54.073
      101 jeremy.demlow@snowflake.com                     Daily lift wait time summary           Daily       True           2 2025-12-03 16:53:04.407 2025-12-03 16:50:19.105
        1          test@skiresort.com                     Daily lift wait time summary           Daily      False           0                     NaT 2025-12-03 16:44:55.431


In [ ]:
# Delete a specific alert by ID
# Change the ALERT_ID below to delete a specific alert

ALERT_ID_TO_DELETE = 201  # Set to an ID like 101 to delete

if ALERT_ID_TO_DELETE:
    result = session.sql(f"""
        UPDATE SKI_RESORT_DB.AGENTS.SCHEDULED_ALERTS
        SET IS_ACTIVE = FALSE
        WHERE ALERT_ID = {ALERT_ID_TO_DELETE}
    """).collect()
    print(f"✅ Deactivated alert {ALERT_ID_TO_DELETE}")
else:
    print("ℹ️ Set ALERT_ID_TO_DELETE to deactivate an alert")


ℹ️ Set ALERT_ID_TO_DELETE to deactivate an alert


In [22]:
# Unsubscribe by keyword (partial match)
# This matches any alert containing the keyword in the question

KEYWORD_TO_UNSUBSCRIBE = None  # Set to something like "lift" or "wait"

if KEYWORD_TO_UNSUBSCRIBE:
    result = session.call('SKI_RESORT_DB.AGENTS.UNSUBSCRIBE_ALERT', 
                          TEST_EMAIL, KEYWORD_TO_UNSUBSCRIBE)
    print(f"📭 {result}")
else:
    print("ℹ️ Set KEYWORD_TO_UNSUBSCRIBE to cancel matching alerts")


ℹ️ Set KEYWORD_TO_UNSUBSCRIBE to cancel matching alerts


In [ ]:
# BULK CLEANUP - Deactivate all alerts for a user
# ⚠️ Use with caution!

CLEANUP_ALL_FOR_USER = False  # Set to True to deactivate all your alerts

if CLEANUP_ALL_FOR_USER:
    result = session.sql(f"""
        UPDATE SKI_RESORT_DB.AGENTS.SCHEDULED_ALERTS
        SET IS_ACTIVE = FALSE
        WHERE USER_EMAIL = '{TEST_EMAIL}'
          AND IS_ACTIVE = TRUE
    """).collect()
    count = session.sql(f"SELECT ROW_COUNT()").collect()[0][0]
    print(f"🧹 Deactivated all alerts for {TEST_EMAIL}")
else:
    print("ℹ️ Set CLEANUP_ALL_FOR_USER = True to deactivate all your alerts")


---
## Dynamic Content with Cortex Analyst

Instead of hardcoded HTML, let Snowflake Intelligence generate insights dynamically for ANY question!


In [23]:
# Call Cortex Analyst to generate dynamic insights
# This works in Snowflake Notebooks - generates insights for ANY question!

def get_cortex_insights(question: str) -> dict:
    """Call Cortex Analyst and get dynamic insights."""
    try:
        import _snowflake
        
        response = _snowflake.send_snow_api_request(
            "POST",
            "/api/v2/cortex/analyst/message",
            {},
            {},
            {
                "messages": [{"role": "user", "content": [{"type": "text", "text": question}]}],
                "semantic_model_file": "@SKI_RESORT_DB.SEMANTIC.STAGE/sem_operations.yaml",
            },
            {},
            30000,
        )
        
        resp_json = json.loads(response["content"])
        
        result = {'question': question, 'sql': None, 'text': None, 'data': None}
        
        if 'message' in resp_json and 'content' in resp_json['message']:
            for content in resp_json['message']['content']:
                if content.get('type') == 'sql':
                    result['sql'] = content.get('statement', '')
                elif content.get('type') == 'text':
                    result['text'] = content.get('text', '')
        
        # Execute SQL if we got one
        if result['sql']:
            df = session.sql(result['sql']).to_pandas()
            result['data'] = df
            
        return result
        
    except Exception as e:
        return {'error': str(e), 'question': question}

# This only works in Snowflake Notebooks, not locally
print("ℹ️ Use get_cortex_insights(question) in Snowflake Notebooks to generate dynamic content")


ℹ️ Use get_cortex_insights(question) in Snowflake Notebooks to generate dynamic content


In [24]:
# Generate dynamic email from Cortex response
# This function converts Cortex Analyst output into a beautiful email

def generate_dynamic_email(question: str, analyst_text: str, data_df=None) -> str:
    """Convert Cortex Analyst response to HTML email."""
    timestamp = datetime.now().strftime("%B %d, %Y at %I:%M %p")
    
    # Convert markdown-ish text to HTML
    html_content = analyst_text.replace('\n', '<br>') if analyst_text else "No insights available."
    
    # Build data table if we have data
    table_html = ""
    if data_df is not None and len(data_df) > 0:
        cols = data_df.columns.tolist()
        header = "".join(f"<th>{c}</th>" for c in cols)
        rows = ""
        for _, row in data_df.head(10).iterrows():
            rows += "<tr>" + "".join(f"<td>{row[c]}</td>" for c in cols) + "</tr>"
        table_html = f"""
        <h4 style="margin-top:20px;">📈 Data</h4>
        <table style="width:100%; border-collapse:collapse; font-size:13px;">
            <thead><tr style="background:#f5f5f5;">{header}</tr></thead>
            <tbody>{rows}</tbody>
        </table>
        """
    
    return f"""
<!DOCTYPE html>
<html>
<head>
    <style>
        body {{ font-family: -apple-system, BlinkMacSystemFont, sans-serif; margin:0; padding:0; background:#f5f5f5; }}
        .container {{ max-width:600px; margin:20px auto; background:white; border-radius:12px; box-shadow:0 2px 8px rgba(0,0,0,0.1); }}
        .header {{ background:linear-gradient(135deg, #1e88e5, #1565c0); padding:25px; color:white; border-radius:12px 12px 0 0; }}
        .content {{ padding:25px; }}
        .insights {{ background:#e3f2fd; padding:20px; border-radius:8px; border-left:4px solid #1976d2; line-height:1.6; }}
        table th {{ padding:8px; text-align:left; border-bottom:2px solid #ddd; }}
        table td {{ padding:8px; border-bottom:1px solid #eee; }}
        .footer {{ background:#fafafa; padding:15px; font-size:12px; color:#666; text-align:center; border-radius:0 0 12px 12px; }}
    </style>
</head>
<body>
    <div class="container">
        <div class="header">
            <h1 style="margin:0; font-size:18px;">🎿 {question[:60]}</h1>
            <div style="opacity:0.9; font-size:13px; margin-top:8px;">{timestamp}</div>
        </div>
        <div class="content">
            <div class="insights">
                <h3 style="margin-top:0; color:#1565c0;">📊 AI Insights</h3>
                {html_content}
            </div>
            {table_html}
        </div>
        <div class="footer">
            Ski Resort Analytics | Powered by Snowflake Cortex
        </div>
    </div>
</body>
</html>
"""

print("✅ Dynamic email generator ready - pass Cortex Analyst output to generate_dynamic_email()")


✅ Dynamic email generator ready - pass Cortex Analyst output to generate_dynamic_email()


In [27]:
# FULL WORKFLOW: Ask ANY question -> Get Cortex insights -> Send email
# Run this in Snowflake Notebooks for dynamic content!

ANY_QUESTION = "What were the busiest lifts yesterday?"

# In Snowflake Notebooks, this would call Cortex Analyst:
insights = get_cortex_insights(ANY_QUESTION)
# html = generate_dynamic_email(ANY_QUESTION, insights['text'], insights.get('data'))

# For local testing, we can simulate with direct SQL:
print(f"🔍 Question: {ANY_QUESTION}")

# Get data dynamically based on question keywords
if "busiest" in ANY_QUESTION.lower() or "lift" in ANY_QUESTION.lower():
    query = """
        SELECT LIFT_NAME, COUNT(*) as TOTAL_RIDES, 
               ROUND(AVG(WAIT_TIME_MINUTES),1) as AVG_WAIT
        FROM SKI_RESORT_DB.MARTS.FACT_LIFT_SCANS f
        JOIN SKI_RESORT_DB.MARTS.DIM_LIFT l ON f.LIFT_KEY = l.LIFT_KEY
        JOIN SKI_RESORT_DB.MARTS.DIM_DATE d ON f.DATE_KEY = d.DATE_KEY
        WHERE d.FULL_DATE = DATEADD(day, -1, CURRENT_DATE())
        GROUP BY LIFT_NAME ORDER BY TOTAL_RIDES DESC LIMIT 10
    """
elif "revenue" in ANY_QUESTION.lower():
    query = """
        SELECT SUM(PURCHASE_AMOUNT) as TOTAL_REVENUE, COUNT(*) as TRANSACTIONS
        FROM SKI_RESORT_DB.MARTS.FACT_TICKET_SALES f
        JOIN SKI_RESORT_DB.MARTS.DIM_DATE d ON f.PURCHASE_DATE_KEY = d.DATE_KEY
        WHERE d.FULL_DATE = DATEADD(day, -1, CURRENT_DATE())
    """
else:
    query = """SELECT 'Use Cortex Analyst in Snowflake Notebooks for dynamic queries' as MESSAGE"""

result_df = session.sql(query).to_pandas()
print(f"\n📊 Results:")
print(result_df.to_string(index=False))


🔍 Question: What were the busiest lifts yesterday?

📊 Results:
           LIFT_NAME  TOTAL_RIDES  AVG_WAIT
      Summit Express          840       1.7
          Family Fun          709       2.0
        Mid Mountain          693       2.0
             Cruiser          664       1.9
         Eagle Ridge          616       1.7
            Blue Sky          538       1.8
            Sunshine          525       1.8
           East Side          507       1.8
         South Ridge          498       1.9
Terrain Park Express          444       1.9


---
## Summary

**This notebook provides:**

1. **Alert Subscription** - Subscribe to any question
2. **Alert Management** - View all, delete by ID, bulk cleanup
3. **Real Data Queries** - Get actual data from your resort
4. **Dynamic Content** - Use Cortex Analyst for AI-generated insights
5. **Email Delivery** - Send formatted emails via Snowflake

**For Production with Snowflake Intelligence:**
- Use `get_cortex_insights(question)` to get dynamic AI responses
- Use `generate_dynamic_email()` to convert to beautiful emails
- The system handles ANY question, not just hardcoded ones!

**Alert Management Quick Reference:**
```python
# View your alerts
session.call('SKI_RESORT_DB.AGENTS.VIEW_MY_ALERTS', email)

# Unsubscribe
session.call('SKI_RESORT_DB.AGENTS.UNSUBSCRIBE_ALERT', email, 'keyword')

# Bulk cleanup
session.sql("UPDATE SCHEDULED_ALERTS SET IS_ACTIVE=FALSE WHERE USER_EMAIL='...'")
```
